# Download Meteor M2 satellite info and calculate times for recording

Requires an API key for n2yo.com and username for geonames.org

In [1]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.1     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   1.0.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
library(httr)

In [3]:
library(lubridate)


Attaching package: ‘lubridate’

The following object is masked from ‘package:base’:

    date



In [4]:
library(jsonlite)


Attaching package: ‘jsonlite’

The following object is masked from ‘package:purrr’:

    flatten



In [5]:
library(yaml)

Source to functions in Functions.ipynb 

In [6]:
eval(parse(text = system2('jupyter', c('nbconvert', 'Functions.ipynb', '--to=script', '--stdout'), stdout = TRUE)))

In [7]:
paste("Running at ", Sys.time() %>% .POSIXct("GMT"), "GMT")

[1] "Running at  2019-09-29 14:18:48 GMT"

# Config

In [8]:
config_list <- tryCatch({
    read_yaml("configuration.yaml")
}, warning = function(w) {
    print("No configuration data found. Run Calibrate_receiver.ipynb")
    stop()
}, error = function(e) {
    print("No configuration data found. Run Calibrate_receiver.ipynb")
    stop()
    }
)

# Load previous satellite orbital data

In [9]:
satellite_data_df <- tryCatch({
    readRDS("satellite_data_df.rds")}, 
    warning = function(w) {
        print("No prior satellite orbital data found on disk")
        tibble(satellite=character(),
            startAz=double(),
            startAzCompass=character(),
            startUTC=as.POSIXct(character()),
            maxAz=double(),
            maxAzCompass=character(),
            maxEl=double(),
            maxUTC=integer(),
            endAz=double(),
            endAzCompass=character(),
            endUTC=as.POSIXct(character()),
            startDate=character(),
            startTime=character(),
            endDate=character(),
            endTime=character(),
            localStartTime=character(),
            duration=double())
}, error = function(e) {
        print("There was an error in trying to load satellite orbital data from disk")
        tibble(satellite=character(),
            startAz=double(),
            startAzCompass=character(),
            startUTC=as.POSIXct(character()),
            maxAz=double(),
            maxAzCompass=character(),
            maxEl=double(),
            maxUTC=integer(),
            endAz=double(),
            endAzCompass=character(),
            endUTC=as.POSIXct(character()),
            startDate=character(),
            startTime=character(),
            endDate=character(),
            endTime=character(),
            localStartTime=character(),
            duration=double())
    }
)

Get rid of passes that have already occurred

In [10]:
current_time_utc <- Sys.time() %>% .POSIXct("GMT")

In [11]:
satellite_data_df <- satellite_data_df %>% filter(startUTC > current_time_utc)

# Update satellite orbital data

##  Using the https://www.n2yo.com/api/ webservice since I can't find any R packages to predict satellite passes locally

Request: /radiopasses/{id}/{observer_lat}/{observer_lng}/{observer_alt}/{days}/{min_elevation}
```
Parameter       Type	Required	Comments
id              integer Yes	        NORAD id
observer_lat	float   Yes	        Observer's latitide (decimal degrees format)
observer_lng	float   Yes	        Observer's longitude (decimal degrees format)
observer_alt	float   Yes	        Observer's altitude above sea level in meters
days            integer Yes	        Number of days of prediction (max 10)
min_elevation   integer Yes	        The minimum elevation acceptable for the highest altitude point of the pass (degrees)
```

Generate API access urls for each satellite based on its NORAD ID

In [12]:
n2yo_urls <- generateN2YOURL(config_list)

Try to safely access the API

In [13]:
satellite_data <- n2yo_urls %>% map(possibly_get_data, 8) %>% set_names(names(config_list$satellites))

In [14]:
satellite_data %>% map(testResponse)

[1] "Successfully fetched data"
[1] "Successfully fetched data"
[1] "Successfully fetched data"
[1] "Successfully fetched data"
[1] "Successfully fetched data"


$METEOR_M2
[1] "Successfully fetched data"

$METEOR_M2_2
[1] "Successfully fetched data"

$NOAA_15
[1] "Successfully fetched data"

$NOAA_18
[1] "Successfully fetched data"

$NOAA_19
[1] "Successfully fetched data"

Extract the payload from the API response

In [15]:
satellite_passes <- satellite_data %>% map(function (x) fromJSON(rawToChar(x$content)) %>% pluck("passes"))

Bind satellite info from the satellites_df dataframe to each row of the orbital data

In [16]:
new_satellite_data_df <- map2(names(satellite_passes), satellite_passes, 
                          function(x,y) cbind(satellite=x, y, stringsAsFactors=FALSE)) %>% 
                          bind_rows %>% full_join(bind_rows(config_list$satellites, .id="satellite"))

Joining, by = "satellite"


Format the start and stop times in UTC

In [17]:
new_satellite_data_df <- new_satellite_data_df %>% mutate(startUTC = as_datetime(startUTC, tz = "UTC")) %>%
    mutate(endUTC = as_datetime(endUTC, tz = "UTC"))

### Filter out passes that we've already downloaded so that they're not duplicated when we bind the new data to the old we've loaded from disk

In [18]:
new_satellite_data_df <- anti_join(new_satellite_data_df, satellite_data_df, by = c("satellite", "startUTC"))

For recording passes need start date, start time and duration. Start time is in the format HH:MM for scheduling with `systemd`. Duration is how long `rtl_fm` needs to run (in **minutes**) from start to shut-down.

In [19]:
new_satellite_data_df <- new_satellite_data_df %>% 
    mutate(localStartDate = format(as_datetime(startUTC, tz = config_list$localTimeZone), "%Y-%m-%d")) %>%
    mutate(localStartTime = format(as_datetime(startUTC, tz = config_list$localTimeZone), "%H:%M")) %>% 
    mutate(duration = as.numeric(ceiling(endUTC - startUTC)))

In [20]:
satellite_data_df <- bind_rows(satellite_data_df, new_satellite_data_df) %>% 
    select('satellite', 'frequency', 'norad_id', 'priority', everything()) %>%
    arrange(localStartDate, localStartTime)

In [21]:
print(paste0("Orbital data for ", nrow(satellite_data_df), " future satellite passes saved"))

[1] "Orbital data for 137 future satellite passes saved"


In [22]:
saveRDS(satellite_data_df, "satellite_data_df.rds")